<a href="https://colab.research.google.com/github/DanielOlson/CompBioAsia/blob/main/CompBioAsia_affinity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Binding Affinity Prediction!
We're now dangerous and ready to do REAL machine learning on REAL problems. Good luck.

#### Setup
Normal setup cells, run each one once. NOTE: the URL should not be "2hm" but instead "2mh". Switch the m and h around to download the file.

In [ ]:
!wget https://umt.box.com/shared/static/2hmfpjcnprr1jt5rik45edghdy0rnw0p.zip
!unzip *.zip

In [ ]:
import torch
from torch import nn
import matplotlib.pyplot as plt
import torch.nn.functional as F

In [ ]:
def read_all_data(indx_file, torch_file, cutoff=30.0, clean=False):
  names = []
  torch_data = torch.load(torch_file)
  with open(indx_file, 'r') as file:
    for line in file:
      line = line.split(' ')
      if len(line) > 0:
        names.append((line[0], line[1]))
  
  ones = torch_data[:,-1] <= cutoff
  zeros = torch_data[:,-1] > cutoff

  torch_data[ones,-1] = 1.0
  torch_data[zeros,-1] = 0.0

  zero_field = torch.zeros_like(torch_data)
  if clean:
    for i in range(17):
      zero_field[torch_data[:,i] == 0,i] = 1
      torch_data[torch_data[:,i] != 0,i] = torch_data[torch_data[:,i] != 0,i] - torch.mean(torch_data[torch_data[:,i] != 0,i], dim=0)
      torch_data[torch_data[:,i] != 0,i] = torch_data[torch_data[:,i] != 0,i] / torch.std(torch_data[torch_data[:,i] != 0,i], dim=0)
  
    torch_data = torch.cat([zero_field, torch_data], dim=-1)

  return names, torch_data

def train_with_data(x,
                    model, batch_size, 
                    steps, learning_rate, 
                    loss_function, checkin=100):
  dev = 'cpu'
  if torch.cuda.is_available():
    dev = 'cuda:0'

  model.to(dev)
  optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

  if batch_size % 2 == 1:
    print("Batch size must be divisible by 2")
    return -1

  pos_x = x[x[:,-1] > 0.5]
  neg_x = x[x[:,-1] < 0.5]

  half_batch = int(batch_size / 2)

  for step in range(steps):
    pbatch_i = torch.randint(0, len(pos_x), (half_batch,))
    nbatch_i = torch.randint(0, len(neg_x), (half_batch,))

    batch_y = pos_x[pbatch_i,-1].view(-1, 1).to(dev)
    batch_x = pos_x[pbatch_i,:-1].to(dev)

    out = model(batch_x)
    loss = loss_function(out, batch_y)

    batch_y = neg_x[nbatch_i,-1].view(-1, 1).to(dev)
    batch_x = neg_x[nbatch_i,:-1].to(dev)

    out = model(batch_x)
    loss += loss_function(out, batch_y)
    

    loss.backward()
    optimizer.step()
    optimizer.zero_grad()
    if step % checkin == 0:
      print('progress: {:.2%} '.format(step / steps), 'loss:', float(loss))

  model.to('cpu')

# This function just tests our accuracy
def test_accuracy(x, model, batch_size=512):
  dev = 'cpu'
  if torch.cuda.is_available():
    dev = 'cuda:0'

  model.to(dev)

  y = x[:,-1].view(-1, 1)
  x = x[:,:-1]

  x = x.to(dev)
  y = y.to(dev)

  pos_correct = 0
  pos_incorrect = 0
  neg_correct = 0
  neg_incorrect = 0
  with torch.no_grad():

    batches = int(len(x) / batch_size) - 1# we lose a little bit of data but thats ok

    for batch in range(batches):
      start_idx = batch * batch_size

      batch_x = x[start_idx:start_idx+batch_size]
      batch_y = y[start_idx:start_idx+batch_size]

      out = model(batch_x)
      out = out > 0.5

      pos_correct += torch.sum((out == batch_y)[batch_y > 0.5])
      pos_incorrect += torch.sum((out != batch_y)[batch_y > 0.5])

      neg_correct += torch.sum((out == batch_y)[batch_y < 0.5])
      neg_incorrect += torch.sum((out != batch_y)[batch_y < 0.5])

     

  model.to('cpu')
  return float(pos_correct / (pos_correct + pos_incorrect)), float(neg_correct / (neg_correct + neg_incorrect)), float((pos_correct + neg_correct) / (pos_correct + neg_correct + pos_incorrect + neg_incorrect))

def pct_ones(data):
  return torch.sum(data[:,-1] == 1.0) / len(data)


dude_names, dude_data = read_all_data('./affinity_data/dude.indx', 
                                      './affinity_data/dude.torch')

litpcba_names, litpcba_data = read_all_data('./affinity_data/litpcba.indx', 
                                          './affinity_data/litpcba.torch')


print('DUD-E Shape:', dude_data.shape)
print('Lit-PCBA Shape:', litpcba_data.shape)

print('DUD-E percentage active: {:.4%}'.format(pct_ones(dude_data)))
print('Lit-PCBA percentage active: {:.4%}'.format(pct_ones(litpcba_data)))

#### ITS TIME !!


---

This problem is a lot harder than MNIST. Below is the code needed to train on the DUD-E dataset. It looks a lot like the MNIST code, but it will be more difficult to get traction.

1.   Making the network deeper (more layers).
2.   Making the network wider (more neurons).
3.   Try bigger and smaller batch sizes.
4.   Try more/less steps.
5.   Using different activation functions (replacing nn.Tanh) such as nn.Sigmoid(), nn.ReLU(), nn.ELU(). a complete list of activation functions can be found at https://pytorch.org/docs/stable/nn.html#non-linear-activations-weighted-sum-nonlinearity. 
6.   Try using different loss functions (Maybe give nn.BCELoss() a go)

In [ ]:

batch_size = 512 # How many images we train on at every step
steps = 20000 # How many total steps we will train for
learning_rate = 0.0001 # How fast we adjust gradients with gradient descent
loss_function = nn.MSELoss() # Which loss function we're using
checkin = int(steps / 5) # How often we print our loss (smaller = more frequently)

# Change this model to change try and improve results
model = nn.Sequential(nn.Linear(17, 5), 
                      nn.Tanh(),
                      nn.Linear(5, 1),
                      nn.Sigmoid()) # <--- the model needs to end with this

# Here we get a starting testing accuracy
# Accuracy is reported as (positive accuracy, negative accuracy, total accuracy)
pos_acc, neg_acc, tot_acc = test_accuracy(dude_data, model)
print("Starting DUD-E accuracy: ({:.4%}, {:.4%} = {:.4%})".format(pos_acc, neg_acc, tot_acc))

print("Training on DUD-E...")
# Train our model with our train_with_data helper function
train_with_data(dude_data,
                model, batch_size, steps,
                learning_rate, loss_function, checkin=checkin)

# Check the ending testing accuracy
pos_acc, neg_acc, tot_acc = test_accuracy(dude_data, model)
print("Ending DUD-E accuracy: ({:.4%}, {:.4%} {:.4%})".format(pos_acc, neg_acc, tot_acc))


In [ ]:
# Test how we do on LitPCBA (we dont' do great)

pos_acc, neg_acc, tot_acc = test_accuracy(litpcba_data, model)
print("litPCBA: ({:.4%}, {:.4%} {:.4%})".format(pos_acc, neg_acc, tot_acc))


In [ ]:
# Show distributions of inputs

r = torch.randint(0, len(dude_data), (10000,))
x = torch.arange(len(r))

for i in range(18):
  print(i)
  y, _ = torch.sort(dude_data[r, i])
  plt.scatter(x, y)
  plt.show()